# Global registration with RANSAC
We are going to use open3d (http://www.open3d.org/) to handle  pointclouds and generation of pointclouds

So make sure to call **pip install open3d**


In [40]:
import open3d as o3d
import numpy as np
import copy

# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

We need to read in our pointclouds.

In [41]:
source = o3d.io.read_point_cloud("ICP/r1.pcd")
#target = o3d.io.read_point_cloud("ICP/r2.pcd")
target = o3d.io.read_point_cloud("ICP/r3.pcd")

# Show models side by side
draw_registrations(source, target)

### Finding features in pointclouds
When working on point clouds it can be benefitial work on a downsampled version of the point cloud.
you can use [```pointcloudname.voxel_down_sample()```](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html) where pointcloud is the name of your point cloud object.

We also need to estimate the normals of the pointcloud points using [```pointcloudname.estimate_normals()```](http://www.open3d.org/docs/latest/python_api/open3d.geometry.PointCloud.html)

And finally find fpfh features or correspondance of the downsampled point clouds.
[```o3d.registration.compute_fpfh_feature()```](http://www.open3d.org/docs/latest/python_api/open3d.registration.compute_fpfh_feature.html#open3d.registration.compute_fpfh_feature)


In [ ]:
voxel_size=0.05

In [42]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [47]:
# Used for downsampling.
voxel_size=0.05 #越大越稀疏

source_down,source_fpfh = preprocess_point_cloud(source,voxel_size)
target_down, target_fpfh = preprocess_point_cloud(target,voxel_size)
draw_registrations(source_down, target_down,recolor = True)

:: Downsample with a voxel size 0.200.
:: Estimate normal with search radius 0.400.
:: Compute FPFH feature with search radius 1.000.
:: Downsample with a voxel size 0.200.
:: Estimate normal with search radius 0.400.
:: Compute FPFH feature with search radius 1.000.


### Ransac
We will now attempt to use ransac to do a global registration of the two poinclouds.

Using the function [```o3d.registration.registration_ransac_based_on_feature_matching```](http://www.open3d.org/docs/latest/python_api/open3d.registration.registration_ransac_based_on_feature_matching.html#open3d.registration.registration_ransac_based_on_feature_matching) from open3d


Try to find the transformation from r1 to r2.
Attempt with point to point and point to plane
```Python
point_to_point =  o3d.registration.TransformationEstimationPointToPoint(False)
point_to_plane =  o3d.registration.TransformationEstimationPointToPlane()
```

When using ransac focus on the arguments below the rest are optional
```Python
ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    distance_threshold,
    point_to_point)
```

In [44]:
	
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)

    point_to_point =  o3d.registration.TransformationEstimationPointToPoint(False)
    point_to_plane =  o3d.registration.TransformationEstimationPointToPlane()

    corr_length = 0.9
    distance_threshold = voxel_size * 1.5

    c0 = o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
    c1 = o3d.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    c2 = o3d.registration.CorrespondenceCheckerBasedOnNormal(0.095)

    checker_list = [c0,c1,c2]

    result = o3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, 
        source_fpfh, target_fpfh,
        distance_threshold,
        point_to_point,
        #point_to_plane,
        checkers = checker_list)
    return result


In [45]:
result_ransac = execute_global_registration(source_down, target_down, source_fpfh,target_fpfh, voxel_size=0.05)

draw_registrations(source_down, target_down, transformation = result_ransac.transformation, recolor = True)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.


## Exercises
### A)
    Can you get a decent transformation from r1 to r3?
### B)
    with the following checkers can you get better results from RANSAC? 
    Try tweaking the parameters of them Can you make Point to Plane work do not spend too long on this if you cant skip it. (I was not able to get a good fit)

You can also try tweaking the voxel_size
```Python
corr_length = 0.9
distance_threshold = voxel_size * 1.5

c0 = o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
c1 = o3d.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
c2 = o3d.registration.CorrespondenceCheckerBasedOnNormal(0.095)

cecker_list = [c0,c1,c2]

ransac_result = o3d.registration.registration_ransac_based_on_feature_matching(
    source_sample, target_sample, 
    source_fpfh, target_fpfh, 
    distance_threshold,
    point_to_point
    checkers = checker_list)
```
